# Understanding SB 50 in LA, Part 1

For this project, we are going to do our best to recreate the analysis performed by Urban Footprint and summarized in [this blog post](https://urbanfootprint.com/analyzing-sb-50/).

Before getting started, make sure that you have the following packages installed: `pandas`, `geopandas`, `numpy`

In [ ]:
# TODO: Import pandas, geopandas, numpy


### Pre-Exercise
During the ITS Data Camp Course, we introduced you to the `folium` Python package to display interactive maps right in your browser, made possible because the folium package builds off the leaflet javascript package. For this exercise, we are going to continue to interactively explore spatial data, but we are going to use a different (but very similar) Python package called `ipyleaflet`, which has some additional functionality and plays a little bit better with GeoPandas DataFrames. Take a moment to read more about `ipyleaflet` [here](https://blog.jupyter.org/interactive-gis-in-jupyter-with-ipyleaflet-52f9657fa7a). You will notice that the article mentions `folium` toward the end in a comparison of similar packages.

##### Install & Setup of ipyleaflet
Follow the instructions (using the conda install) on the [documentation page](https://ipyleaflet.readthedocs.io/en/latest/installation.html#jupyterlab-extension). If you are having difficulty getting the map to appear in your notebook (with everything else loading correctly), take a look at this [SO question](https://gis.stackexchange.com/questions/303261/ipyleaflet-map-object-doesnt-display-in-jupyter-notebook-but-it-gets-created) for the command to enable the extension. You will run this command not in the notebook, but instead in the command prompt (OSX) / anaconda prompt (Windows).

##### Test the Install
We can confirm that the package was installed properly and the extension was enabled by running the following code, which should display a map with a single point in the middle.

In [ ]:
from ipyleaflet import Map, Marker

center=(34.052235,-118.243683)
m = Map(center=center, zoom=10)
marker = Marker(location=center, draggable=True)
m.add_layer(marker);

m

## Transportation-Based Criteria
Let's focus exclusively on service provided by LA Metro for now. There are two different ways that high-quality transit areas were defined in the legislation:
1. 1/2 mile from any major rail stop
2. 1/4 mile of any high-quality bus stop

### Major Rail Stops

##### Step 1: Import the Metro Rail Stops
Let's start by getting the Metro Rail stops from the [LA City GeoHub](http://geohub.lacity.org/datasets/6679d1ccc3744a7f87f7855e7ce33395_1?geometry=-119.297%2C33.719%2C-117.230%2C34.118). Although we could simply download the data and import it into our folder, let's practice calling the API to get the data. Once you make a successful call, save it to our `data/raw` directory.

In [ ]:
# TODO: Import requests & json packages

# TODO: Call the API to get the GeoJSON data, and save to 'data/raw'


##### Step 2: Convert to GeoPandas DataFrame
Now that we have our data saved to disk, let's go ahead and read it back in and convert it to a GeoPandas dataframe. Because we are going to perform spatial manipulation, we need to make sure we first explicitly state our CRS and then convert it to an appropriate CRS for perform spatial manipulations (I recommend using NAD-83). Take a look at the GeoPandas documentation [here](http://geopandas.org/projections.html) for setting projections and re-projecting data.

_For users experiencing an issue with projections: It is likely that somehow your conda installation provided you an older version of pyproj (which you can check). You may need to remove `pyproj` via `conda remove --force pyrpoj` and then reinstall it using pip to get pyproj 2.4.x_ 

In [ ]:
# TODO: Read in to GeoPandas Dataframe
rail_stops_gdf = 

In [ ]:
# TODO: Examine the DF head to make sure it read in correctly
rail_stops_gdf.head()

In [ ]:
# TODO: Examine current CRS


# TODO: Re-Project to NAD83
rail_stops_nad83 = 

##### Step 3: Buffer 1/2 mile
Now that we've re-projected our data, we can go ahead and perform spatial manipulations. Let's create a 1/2 mile buffer around each of the rail stops. _(Pro tip: Each CRS has it's own unit of measurement. Make sure you keep track of what you are currently using!)_ Once we do this, we are going to project it back to WGS84 so that we display it in our map below. We will also want to make sure we save to disk at `data/processed` so that we have it if needed later. 

In [ ]:
# TODO: buffer 1/2 mi
# You could have used either ft or m, as long as it aligns with your CRS
rail_stops_halfmi_buff = 

# TODO: Add original rail data to buffer
rail_stops_nad83.geometry = 

# TODO: Re-project back to WGS84 for map display
rail_stop_buff_wgs84 = 

# TODO: Write out to data/processed


In [ ]:
# View the head of the table and confirm that the geometry is showing POLYGON instead of POINT
rail_stop_buff_wgs84.head()

Let's confirm that the buffer looks correct by displaying it (along with the rail stops) on the map, following the example [here](https://ipyleaflet.readthedocs.io/en/latest/api_reference/geodata.html). You should see circle polygons that look to have a radius of about 1/2 mi.  

In [ ]:
# TODO: Display on map

Nice work! Let's move on to high-quality bus corridors.

### High Quality Bus Corridors
In addition to looking at neighborhoods surrounding rail stops, the legislation also allows for additional density near "high-quality bus stops". We will use Urban Footprint's definition of High Quality Bus Corridor as having the following criteria:
- less than a 10-minute average headway for three consecutive hours during the morning peak, from 6 to 10 AM, and the evening peak, from 3 to 7 PM.
- maintain an average headway below 20 minutes over the course of a weekday from 6 AM to 10 PM and an average headway below 30 minutes on weekends
  
Because this definition is a bit trickier, we are going to look a bit deeper through Metro's GTFS data. GTFS is a standard developed by Google showing scheduled _(not realtime)_ data. [TransitWiki](https://www.transitwiki.org/TransitWiki/index.php/General_Transit_Feed_Specification) and the [Google developers page](https://developers.google.com/transit/gtfs/) both provide a good overview of the GTFS standard; take some time to look through the documentation on each. Metro provides it's own GTFS data for easy consumption [here](http://developer.metro.net/gtfs-schedule-data/), with a README containing information on each archive.

##### Working with GTFS Data
There have been numerous development efforts (in multiple programming languages) that make it easier to ingest, analyze, view, and produce GTFS data, many of which you can see [here](https://github.com/CUTR-at-USF/awesome-transit). We will one of the [GTFS libraries](https://github.com/CUTR-at-USF/awesome-transit#gtfs-libraries) to help with the ingestion of our data, but the rest we will do using `GeoPandas`. For this exercise we will be specifically focused on calculating frequency, but feel free to also explore the myriad of tools that have been developed to look at GTFS with another objective in mind.

##### Our Plan
Transit agencies _may_ optionally publish a table called `frequencies.txt`, which makes it fairly easy to calculate the average frequency at the peak hours. However, you will notice that Metro's archive doesn't contain that file, which means we will need to calculate it ourselves.

While looking through [Metro's bus GTFS data](http://developer.metro.net/gtfs-schedule-data/), we decide we are interested in 3 tables: 
- `trips`: This specifies which routes are bus routes. This is our starting point.
- `stop_times`: Combined with trips, we will calculate the arrival frequency using the criteria above to determine the High Quality Routes
- `stops`: Once we determine the High Quality Bus Routes, we will want to create the 1/4 mile buffer around the stops

##### Step 1: Install `partridge` package
We will use the `partridge` package for lightweight parsing of GTFS files (to keep memory overhead low) into `GeoPandas` dataframes that we can then work on for further analysis. Follow the install instructions [here](https://github.com/remix/partridge).

##### Step 2: Download Data, Get busiest day
Begin by downloading the entire LA Metro Bus GTFS zipped file to `data/raw/gtfs_bus-master.zip`. You will notice that the entire size of the folder is around 40MB zipped (and 237MB unzipped!). Whlie data of that size is not an issue storing on disk, loading all of it into [memory (RAM)](https://www.backblaze.com/blog/whats-diff-ram-vs-storage/) will be wasteful and unnecessary. We won't need to unzip the file as `partridge` is designed to directly read from zipped directories.

We are going to use `partridge` to get the service IDs that are associated with the busiest day in the GTFS feed.

In [ ]:
# Import partridge package
import partridge as ptg

# TODO: Set inpath to zipped directory
inpath = 

# TODO: follow partridge package quickstart example
#       to get date, service_ids for the busiest day

Let's see what's in this (now filtered) feed.

In [ ]:
# Print contents of the feed
print([i for i in dir(feed)])

In [ ]:
# TODO: Pull stop times from the feed and examine the head of the table
stop_times_df = 

# Examine the head of the table
stop_times_df.head()

In [ ]:
# TODO: Extract the trips and examine the head of the table
trips_df = 


##### Step 3: Write helper function

Write a function that filters the stop_times for only the hours of interest. You'll notice that the times are in seconds, so make sure to account for the appropriate conversion. We will do our calculations based on the `arrival_time`.

In [ ]:
# TODO: Create a function to filter our stop_times dataframe
#       by a beginning hour (inclusive) and end hour (exclusive)
#       using methods in the Pandas library
def filter_stoptimes_byhour(df, begin_hour, end_hour):
    ''' 
    Filter a GTFS DF by start/end hour
    
    :param df DataFrame: Pandas DF of stop_times from partridge
    :param begin_hour int: begin hour (inclusive), 0-23
    :param end_hour int: end hour (not inclusive), 1-24
    returns: Filtered stop_times df
    '''


# Compare the length of raw & filtered dataframes for comparison
print(f'The unfiltered DF contains {len(stop_times_df)} rows.')
print(f'Between 7-10am, there are {len(filter_stoptimes_byhour(stop_times_df, 7, 10))} rows.')

##### Step 4: Calculate Stop Average Frequency at Each Peak
As a reminder, The first criterion for High-Quality Bus Corridors is that the routes "have less than a 10-minute average headway for three consecutive hours" in the AM peak. During this step, we are going to filter for all the routes that have more than 18 scheduled arrivals in the 3 hour period between 7-10am.

With that helper function complete, we are ready to write the aggregation pipeline to compute the average frequency for each hour in the `stop_times` dataframe. _This next step will involve a fairly heavy use of the `pandas` library. I recommend [brushing up](https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html) on some of the most common operations if the steps below seem intimidating. Also, note that the `pandas` library often has multiple methods that essentially perform the same function, so how you complete the next step will look a little different depending on the methods you choose._ 
  
We will calculate the frequency through the following steps:
1. Filter for hours by using our new function `filter_stoptimes_byhour`.
2. Join `stop_times` to `trips`, joining on the `trip_id`.
3. Group by the following:
    - `route_id`
    - `direction_id`
    - `stop_id`
    - `service_id`
4. Summarize by count into a new column called `count`
5. Filter for rows that have `count` >= 18.
6. Create a new column `am_headway` that shows the average headway (in minutes).

In [ ]:
# TODO: Perform aggregation & computation steps from above


# Exmaine the length and head of the new aggregated table
print(len(stop_freq_am_df))
stop_freq_am_df.head()

In addition to providing minimum of a 10 minute headway for the AM Peak, the stop must also meet the same minimum frequency for the PM Peak. Again, although the legislation gives a little bit of leeway with a 4 hour window, we are going to pick the hours of 4-7pm (the same hours used in the UF analysis). The new column should be named `pm_headway`.
  
_Hint: Make sure to use 24 hour format in your `filter_stoptimes_byhour` function_

In [ ]:
# TODO: Perform same steps for PM Peak


# TODO: Exmaine the length and head of the new aggregated table


##### Step 5: Filter for those stops that meet both headway conditions
The stop must meet both AM and PM frequency criteria in order to be eligible for SB50 consideration. If you looked at the length of the two dataframes, you'll see that there is a difference in the number of rows, so we know that getting the intersection of both tables will produce (at most) the length of the smaller dataframe. 

To perform this filter, we need to do the following:
- Use a join to filter for rows with a `route_id` and `stop_id` that meet both criteria (`direction_id` can be different).
- Because a route has multiple directions, it is possible that a route would have frequent service in both directions at the same time, resulting in either of the tables having multiple rows with the same `route_id` and `stop_id`. Performing the join will result in the [cartesian product](https://www.geeksforgeeks.org/sql-join-cartesian-join-self-join/) of the matches. It shouldn't be too many, but make sure that we remove any rows with duplicate (`route_id`, `stop_id`) after the join.

In [ ]:
# TODO: Find route_id, stop_id that meet both criteria


# TODO: Print the length and head of the new table for confirmation


Now that we have the high-frequency stops, we can join it to  the `stops` feed to get the geometry objects for the buffer. Let's keep only the `stop_id`, `stop_name`, `am_headway`, `pm_headway`, `stop_lat`, `stop_lon` columns, dropping the rest.

In [ ]:
# TODO: Pull stops from the feed and examine the head of the table


# TODO: Join stops with stop_freq_df and drop unnecessary columns


# TODO: Exmaine the head of the final df


##### Step 7: Create 1/4 mi. buffers around High-Quality Transit Corridor Stops
After the join, follow the same general steps as with the rail stations to create buffers around each of these stops (but this time for 1/4 mi. instead of 1/2 mi.).



In [ ]:
# TODO: Convert to gdf, project, buffer


Go ahead and display the buffers on a map using `ipyleaflet`.

In [ ]:
# TODO: Create map object, add buffers, and display



Finally, as with the rail stop, let's go ahead and save this buffered geometry to `data/processed/hqbus_stop_buff_wgs84.geojson`.

In [ ]:
# TODO: Write out to data/processed


### To Be Continued...
In the next part, we will add the non-transportation criteria for areas that were considered for SB50. Stay tuned.